## 1. Обучение модели word2vec

In [ ]:
#!pip install gensim

**Gensim** &ndash; библиотека для обработки и векторного представления текстов, в которой реализованы также модели word2vec и fastText. Для демонстрации принципов работы с моделью word2vec в gensim воспользуемся небольшим корпусом текстов `common_texts` из модуля `gensim.test.utils`.

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts

In [ ]:
common_texts

Обучение производится через вызов функции `Word2Vec`, первым аргументом которой является набор текстов, параметр `vector_size` задает размер эмбеддингов, `window` определяет ширину окна при формировании контекстных слов, `min_count` позволяет учитывать только те слова, которые встретились в корпусе не менее определенного количества раз (в примере не менее двух).

In [ ]:
model = Word2Vec(common_texts, vector_size=20, window=3, min_count=2, epochs=1)

Если не указать корпус текстов в модели, то обучение модели производится через метод `train` после вызова функции `build_vocab`, где в качестве аргумента используется корпус. Такой подход позволяет позже дообучать модели.

In [ ]:
model = Word2Vec(vector_size=20, window=3, min_count=2)
model.build_vocab(common_texts)
model.train(common_texts, total_examples=len(common_texts), epochs=1)

После обучения можно получить векторное представление для каждого из слов корпуса, а также вывести наиболее близкие к ним слова:

In [ ]:
model.wv['computer']

In [ ]:
model.wv.most_similar('computer', topn=10)

## 2. fastText

In [ ]:
from gensim.models.fasttext import FastText

Обучение модели `fastText` производится аналогичным образом:

In [ ]:
model = FastText(common_texts, vector_size=4, window=3, min_count=2)

или

In [ ]:
model = FastText(vector_size=4, window=3, min_count=2)
model.build_vocab(common_texts)
model.train(common_texts, total_examples=len(common_texts), epochs=1)

In [ ]:
model.wv.most_similar('computer', topn=10)

## 3. Загрузка предобученных моделей

Gensim позволяет загружать предобученные вектора. Список доступных предобученных моделей доступен в `gensim.downloader`.

In [ ]:
import gensim.downloader

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [ ]:
glove_vectors['computer']

In [ ]:
glove_vectors.most_similar('computer')

Предобученные вектора слов можно использовать в моделях PyTorch.

In [ ]:
fasttext_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

In [ ]:
import torch
import torch.nn as nn

In [ ]:
weights = torch.FloatTensor(fasttext_vectors.vectors)
# embedding = nn.Embedding.from_pretrained(weights)
# embedding.requires_grad = False

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, n_classes, matrix_embeddings):
        super(TextClassifier, self).__init__()
        
        self.input_dim = input_dim
        self.n_classes = n_classes
        self.hidden_dim = 512
        
        self.embeddings = nn.Embedding.from_pretrained(matrix_embeddings)
        self.embeddings.requires_grad = False
        
        self.classifier = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.n_classes))
        
    def forward(self, input_sentence):
        u = self.embeddings(input_sentence)
        return self.classifier(u)